In [5]:
# Hospital Resource Allocation – Phase One (Region-Based Random Assignment)
import pandas as pd
import numpy as np

# === STEP 1: Load datasets ===
sir_df = pd.read_csv("lacounty_covid(2)(1).csv")
sir_df = sir_df[sir_df["Time"] == sir_df["Time"].max()]  # Use most recent prediction
sir_df.rename(columns={"Cases": "Predicted_Infected"}, inplace=True)

hospital_df = pd.read_excel("cleaned icus and ventilators(2).xlsx")
hospital_df.rename(columns={"hospitals": "Hospital"}, inplace=True)

region_map_df = pd.read_csv("Optimised_Allocation_Table(2).csv")  # Contains Region info for Community & Hospital

# === STEP 2: Clean and prepare data ===
hospital_df["Hospital"] = hospital_df["Hospital"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
hospital_df["ICU"] = hospital_df["ICU"].astype(str).str.replace(r"[^\d]", "", regex=True).replace("", "0").astype(int)
hospital_df["Ventilators"] = hospital_df["Ventilators"].astype(str).str.replace(r"[^\d]", "", regex=True).replace("", "0").astype(int)

sir_df["Community"] = sir_df["Community"].astype(str).str.strip()
region_map_df["Community"] = region_map_df["Community"].astype(str).str.strip()
region_map_df["Optimised Hospital"] = region_map_df["Optimised Hospital"].astype(str).str.strip()

# === STEP 3: Merge region info ===
# Get community → region
community_region = region_map_df[["Community", "Region"]].drop_duplicates()
sir_df = sir_df.merge(community_region, on="Community", how="left")

# Get hospital → region
hospital_region = region_map_df[["Optimised Hospital", "Region"]].drop_duplicates()
hospital_region.rename(columns={"Optimised Hospital": "Hospital"}, inplace=True)
hospital_df = hospital_df.merge(hospital_region, on="Hospital", how="left")

# === STEP 4: Estimate demand (3% ICU, 6% ventilator) ===
sir_df["ICU_Demand"] = (sir_df["Predicted_Infected"] * 0.03).round().astype(int)
sir_df["Ventilator_Demand"] = (sir_df["Predicted_Infected"] * 0.06).round().astype(int)

# === STEP 5: Randomly assign each community to a hospital within the same region ===
def assign_random_hospital(region):
    candidates = hospital_df[hospital_df["Region"] == region]["Hospital"].values
    if len(candidates) == 0:
        return np.nan
    return np.random.choice(candidates)

sir_df["Assigned_Hospital"] = sir_df["Region"].apply(assign_random_hospital)

# === STEP 6: Aggregate total demand per hospital ===
demand_summary = sir_df.groupby("Assigned_Hospital")[["ICU_Demand", "Ventilator_Demand"]].sum().reset_index()
demand_summary.rename(columns={"Assigned_Hospital": "Hospital"}, inplace=True)

# === STEP 7: Merge demand with hospital supply ===
hospital_final = hospital_df.merge(demand_summary, on="Hospital", how="left").fillna(0)
hospital_final["ICU_Demand"] = hospital_final["ICU_Demand"].astype(int)
hospital_final["Ventilator_Demand"] = hospital_final["Ventilator_Demand"].astype(int)
hospital_final["ICU_Remaining"] = hospital_final["ICU"] - hospital_final["ICU_Demand"]
hospital_final["Ventilator_Remaining"] = hospital_final["Ventilators"] - hospital_final["Ventilator_Demand"]

# === STEP 8: Save result ===
hospital_final.to_csv("Phase_One_Hospital_Resource_Allocation.csv", index=False)
print("✅ Phase One complete. Output saved as Phase_One_Hospital_Resource_Allocation.csv")


✅ Phase One complete. Output saved as Phase_One_Hospital_Resource_Allocation.csv


In [6]:
# Hospital Resource Allocation – Phase Two (Sorted Matching by Region)
import pandas as pd

# === STEP 1: Load datasets ===
sir_df = pd.read_csv("lacounty_covid(2)(1).csv")
sir_df = sir_df[sir_df["Time"] == sir_df["Time"].max()]
sir_df.rename(columns={"Cases": "Predicted_Infected"}, inplace=True)

hospital_df = pd.read_excel("cleaned icus and ventilators(2).xlsx")
hospital_df.rename(columns={"hospitals": "Hospital"}, inplace=True)

region_map_df = pd.read_csv("Optimised_Allocation_Table(2).csv")  # Contains Community & Hospital to Region mapping

# === STEP 2: Clean and merge region info ===
hospital_df["Hospital"] = hospital_df["Hospital"].astype(str).str.replace(r"\s+", " ", regex=True).str.strip()
hospital_df["ICU"] = hospital_df["ICU"].astype(str).str.replace(r"[^\d]", "", regex=True).replace("", "0").astype(int)
hospital_df["Ventilators"] = hospital_df["Ventilators"].astype(str).str.replace(r"[^\d]", "", regex=True).replace("", "0").astype(int)

sir_df["Community"] = sir_df["Community"].astype(str).str.strip()
region_map_df["Community"] = region_map_df["Community"].astype(str).str.strip()
region_map_df["Optimised Hospital"] = region_map_df["Optimised Hospital"].astype(str).str.strip()

# Merge Region info
community_region = region_map_df[["Community", "Region"]].drop_duplicates()
sir_df = sir_df.merge(community_region, on="Community", how="left")

hospital_region = region_map_df[["Optimised Hospital", "Region"]].drop_duplicates()
hospital_region.rename(columns={"Optimised Hospital": "Hospital"}, inplace=True)
hospital_df = hospital_df.merge(hospital_region, on="Hospital", how="left")

# === STEP 3: Estimate demand ===
sir_df["ICU_Demand"] = (sir_df["Predicted_Infected"] * 0.03).round().astype(int)
sir_df["Ventilator_Demand"] = (sir_df["Predicted_Infected"] * 0.06).round().astype(int)
sir_df["Total_Demand"] = sir_df["ICU_Demand"] + sir_df["Ventilator_Demand"]
hospital_df["Total_Capacity"] = hospital_df["ICU"] + hospital_df["Ventilators"]

# === STEP 4: Sort and match within each region ===
assignments = []

for region in sir_df["Region"].dropna().unique():
    region_communities = sir_df[sir_df["Region"] == region].copy()
    region_hospitals = hospital_df[hospital_df["Region"] == region].copy()

    region_communities = region_communities.sort_values(by="Total_Demand", ascending=False).reset_index(drop=True)
    region_hospitals = region_hospitals.sort_values(by="Total_Capacity", ascending=False).reset_index(drop=True)

    min_len = min(len(region_communities), len(region_hospitals))
    for i in range(min_len):
        community_row = region_communities.loc[i]
        hospital_name = region_hospitals.loc[i, "Hospital"]
        assignments.append({
            "Community": community_row["Community"],
            "Region": region,
            "Predicted_Infected": community_row["Predicted_Infected"],
            "ICU_Demand": community_row["ICU_Demand"],
            "Ventilator_Demand": community_row["Ventilator_Demand"],
            "Assigned_Hospital": hospital_name
        })

# === STEP 5: Aggregate demand to hospitals ===
assigned_df = pd.DataFrame(assignments)
hospital_demand = assigned_df.groupby("Assigned_Hospital")[["ICU_Demand", "Ventilator_Demand"]].sum().reset_index()
hospital_demand.rename(columns={"Assigned_Hospital": "Hospital"}, inplace=True)

# === STEP 6: Merge with hospital capacity ===
hospital_final = hospital_df.merge(hospital_demand, on="Hospital", how="left").fillna(0)
hospital_final["ICU_Demand"] = hospital_final["ICU_Demand"].astype(int)
hospital_final["Ventilator_Demand"] = hospital_final["Ventilator_Demand"].astype(int)
hospital_final["ICU_Remaining"] = hospital_final["ICU"] - hospital_final["ICU_Demand"]
hospital_final["Ventilator_Remaining"] = hospital_final["Ventilators"] - hospital_final["Ventilator_Demand"]

# === STEP 7: Save result ===
hospital_final.to_csv("Phase_Two_Hospital_Resource_Allocation.csv", index=False)
print("✅ Phase Two complete. Output saved as Phase_Two_Hospital_Resource_Allocation.csv")


✅ Phase Two complete. Output saved as Phase_Two_Hospital_Resource_Allocation.csv
